***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-799b  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [1]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 11 12:47 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

### Load Data

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'wiki_preproccess' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

# Building an inverted index

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [4]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [9]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [10]:
from inverted_index_gcp import InvertedIndex

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tf_counter = Counter(tokens)
  for term in all_stopwords: # remove from counter
    tf_counter.pop(term, None)
  word_tuples = [(term, (id, frequency)) for term, frequency in tf_counter.items()]
  return word_tuples

def stemmed_word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [stemmer.stem(token) for token in tokens]
  tf_counter = Counter(tokens)
  for term in all_stopwords: # remove from counter
    tf_counter.pop(term, None)
  word_tuples = [(term, (id, frequency)) for term, frequency in tf_counter.items()]
  return word_tuples
    

def anchor_count(anchors, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  anchor_text = ''
  for anchor in anchors:
    anchor_text += anchor['text'] + ' '
  tokens = [token.group() for token in RE_WORD.finditer(anchor_text.lower())]
  tf_counter = Counter(tokens)
  for term in all_stopwords: # remove from counter
    tf_counter.pop(term, None)
  word_tuples = [(term, (id, frequency)) for term, frequency in tf_counter.items()]
  return word_tuples

def document_length(text):
  return len([token for token in RE_WORD.finditer(text.lower()) if token not in all_stopwords])

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.mapValues(lambda x: len(x))

def calculate_tertm_total(postings):
  return postings.mapValues(lambda x: np.sum([doc[1] for doc in x]))

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(folder, postings, bucket):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  posting_by_hash = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
  posting_loc = posting_by_hash.map(lambda x: InvertedIndex.write_a_posting_list(folder, x, bucket)) 
  return posting_loc

In [12]:
bucket = "wiki_irt_data"
bucket_name = "wiki_irt_data"

### Title Index Builder

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("title", "id").rdd
# Count number of wiki pages
parquetFile.count()

In [14]:
folder = 'title_bins'

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# reduce
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# count term total
term_total = calculate_tertm_total(postings)
term_total_dict = term_total.collectAsMap()
# calculate w to document frequency
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
#_ = partition_postings_and_write(folder, postings, bucket).collect()
print(time() - t_start)

In [16]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket, prefix='title_bins'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Add the Term total 
inverted.term_total = term_total_dict
# write the global stats out
inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

### Body Index Builder

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
# Count number of wiki pages
parquetFile.count()

In [14]:
folder = 'text_bins'

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
# count term total
term_total = calculate_tertm_total(postings_filtered)
term_total_dict = term_total.collectAsMap()
# word to document frequency
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
#_ = partition_postings_and_write(folder, postings_filtered, bucket).collect()
index_const_time = time() - t_start

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket, prefix='text_bins'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Add the Term total 
inverted.term_total = term_total_dict
# write the global stats out
inverted.write_index('.', 'text_index')
# upload to gs
index_src = "text_index.pkl"
index_dst = f'gs://{bucket}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

### Anchor Index Builder

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("anchor_text", "id").rdd
# Count number of wiki pages
parquetFile.count()

In [20]:
folder = 'anchor_bins'

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: anchor_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# count term total
term_total = calculate_tertm_total(postings)
term_total_dict = term_total.collectAsMap()
# calculate df
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(folder, postings, bucket).collect()
index_const_time = time() - t_start

In [22]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket, prefix='anchor_bins'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket}/{index_src}'
!gsutil cp $index_src $index_dst

In [24]:
!gsutil ls -lh $index_dst

  95.1 MiB  2023-01-11T16:37:57Z  gs://wiki_irt_data/anchor_index.pkl
TOTAL: 1 objects, 99721081 bytes (95.1 MiB)


### DL, doc titles, PR, PV

In [13]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
parquetFile.count()

6348910

In [14]:
doc_length = doc_text_pairs.map(lambda x: (x[1], document_length(x[0])))
dl = doc_length.collectAsMap()

In [15]:
index_src = 'doc_length.pkl'
index_dst = f'gs://{bucket}/{index_src}'
with open(index_src, 'wb') as f:
  pickle.dump(dl, f)

!gsutil cp $index_src $index_dst  

Copying file://doc_length.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 44.8 MiB/ 44.8 MiB]                                                
Operation completed over 1 objects/44.8 MiB.                                     


In [16]:
!gsutil ls -lh $index_dst

 44.84 MiB  2023-01-11T15:34:20Z  gs://wiki_irt_data/doc_length.pkl
TOTAL: 1 objects, 47014993 bytes (44.84 MiB)


In [17]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("id", "title").rdd
parquetFile.count()

6348910

In [18]:
doc_title = doc_text_pairs.collectAsMap()

In [ ]:
index_src = 'doc_titles.pkl'
index_dst = f'gs://{bucket}/{index_src}'
with open(index_src, 'wb') as f:
  pickle.dump(doc_title, f)

!gsutil cp $index_src $index_dst   

In [ ]:
!gsutil ls -lh $index_dst

## stemmed indices

In [13]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("title", "id").rdd
# Count number of wiki pages
parquetFile.count()

6348910

In [14]:
folder = 'stemmed_title_bins' 
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))
# reduce
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# count term total
term_total = calculate_tertm_total(postings)
term_total_dict = term_total.collectAsMap()
# calculate w to document frequency
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(folder, postings, bucket).collect()
print(time() - t_start)


224.09971833229065


In [15]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket, prefix='stemmed_title_bins'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
print(len(super_posting_locs))

1663150


In [16]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Add the Term total 
inverted.term_total = term_total_dict
# write the global stats out
inverted.write_index('.', 'stemmed_title_index')
# upload to gs
index_src = "stemmed_title_index.pkl"
index_dst = f'gs://{bucket}/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst

Copying file://stemmed_title_index.pkl [Content-Type=application/octet-stream]...
- [1 files][120.2 MiB/120.2 MiB]                                                
Operation completed over 1 objects/120.2 MiB.                                    
120.23 MiB  2023-01-12T06:50:28Z  gs://wiki_irt_data/stemmed_title_index.pkl
TOTAL: 1 objects, 126067938 bytes (120.23 MiB)


### Body stemmed

In [17]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
# Count number of wiki pages
parquetFile.count()


6348910

In [ ]:
folder = 'stemmed_text_bins'
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
# count term total
term_total = calculate_tertm_total(postings_filtered)
term_total_dict = term_total.collectAsMap()
# word to document frequency
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(folder, postings_filtered, bucket).collect()
index_const_time = time() - t_start


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket, prefix='stemmed_text_bins'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
print(len(super_posting_locs))

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Add the Term total 
inverted.term_total = term_total_dict
# write the global stats out
inverted.write_index('.', 'stemmed_text_index')
# upload to gs
index_src = "stemmed_text_index.pkl"
index_dst = f'gs://{bucket}/{index_src}'
!gsutil cp $index_src $index_dst
!gsutil ls -lh $index_dst

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  links = pages.flatMapValues(lambda x: x)
  edges = links.map(lambda x: (x[0], x[1].id)).distinct()
  vertices = links.map(lambda x: Row(x[1].id)).union(links.map(lambda x: Row(x[0]))).distinct()
  return edges, vertices

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

### Page View

In [ ]:
import os
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

### normalizing pagerank and pageview

In [10]:
def read_index(project, bucket_name, name, bin_dir='.'):
    client = storage.Client(project=project)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f'{name}.pkl')
    file_bytes = blob.download_as_string()
    index = pickle.loads(file_bytes)
    return index

In [20]:
project = 'irt-hw3'
bucket = 'wiki_irt_data'
index_src = 'page_rank'
pr = read_index(project, bucket, index_src)
#!gsutil cp $index_src $index_dst   

In [13]:
max_score = max(pr.values())
max_score

9913.728782160782

In [14]:
normalized_pr = {key: value/max_score for key, value in pr.items()}

In [16]:
max_score = max(normalized_pr.values())
max_score

1.0

In [21]:
index_dst = f'gs://{bucket}/{index_src}.pkl'
with open(f'{index_src}.pkl', 'wb') as f:
  pickle.dump(normalized_pr, f)

!gsutil cp $index_src $index_dst   

Copying file://page_rank [Content-Type=application/octet-stream]...
| [1 files][ 84.7 MiB/ 84.7 MiB]                                                
Operation completed over 1 objects/84.7 MiB.                                     


In [22]:
project = 'irt-hw3'
bucket = 'wiki_irt_data'
index_src = 'page_views'
pr = read_index(project, bucket, index_src)
#!gsutil cp $index_src $index_dst  
max_score = max(pr.values())
print(max_score)
normalized_pr = {key: value/max_score for key, value in pr.items()}
max_score = max(normalized_pr.values())
max_score

181126232


1.0

In [24]:
index_dst = f'gs://{bucket}/{index_src}.pkl'
with open(f'{index_src}', 'wb') as f:
  pickle.dump(normalized_pr, f)

!gsutil cp $index_src $index_dst 

Copying file://page_views [Content-Type=application/octet-stream]...
\ [1 files][143.8 MiB/143.8 MiB]                                                
Operation completed over 1 objects/143.8 MiB.                                    


# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.93 GiB     gs://sise_ir_assignment3/postings_gcp
